## Init 🍄

---

### Dependencies:

In [36]:
import yaml
import re
from datetime import datetime
import pandas as pd

In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [38]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

In [39]:
# edge:object = webdriver.Edge(executable_path="./utils/msedgedriver.exe")
edge = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
edge.maximize_window()

### Functions:

In [40]:
def resources(name:str) -> dict[str:str]:
    """Read xpath.yml file that contains xpath's to scrape"""

    with open("xpath.yml", "r") as f:
        content:dict[str:str] = yaml.safe_load(f)
        return content[name]

In [41]:
def close_prompt(driver:object) -> None:
    """Close ad window"""

    button:str = "//a[@class='btn btn-subscription-light']"
    driver.find_element(by=By.XPATH, value=button).click()

In [42]:
def choose_category(category:str, driver:object) -> None:
    """Choose ad category"""

    driver.find_element(by=By.XPATH, value=category).click()

In [43]:
def click_element(element:object, path:str) -> None:
    """Click element on the current document"""

    button = element.find_element(by=By.XPATH, value=path)
    button.click()

In [44]:
def extract_description(element:object, path:str) -> str:
    """Extract ad description on the current document"""

    description:str = element.find_element(by=By.XPATH, value=path)
    return description

In [45]:
def extract_feature(element:object, path:str) -> list[str]:
    """Extract some feature from list document based on xpath used."""

    feature = []

    try:
        feature:str = element.find_elements(by=By.XPATH, value=path)
        feature = [data_point.text for data_point in feature]
        return feature
    except Exception:
        feature.append("Web wasn't able to scrape the feature")

In [46]:
def persistence(dataframe:pd.DataFrame) -> None:
    """Generate .csv file from data scraped"""

    file_name = datetime.today().strftime("%B-%d")
    dataframe.to_csv(f"persistence/{file_name}.csv", index=False)

In [47]:
def get_len(features:list[list[str]]) -> bool:
    """
    Check if all features has the same length to create new df
    :param features: list of features extracted with scraping
    :return: a list with all lengths where first place is the len of the first element and the mostly accepted.
    """

    base_len = len(features[0])
    lengths = []
    lengths.append(base_len)
    for feature in features:
        if len(feature) == base_len:
            continue
        else:
             lengths.append(len(feature))
    return lengths

In [48]:
def check_len(lengths:list[int]) -> bool:
    """
    Check if all elements in list has the same value.
    :param lengths: list of lengths
    :return: a boolean
    """
    for length in lengths[1:]:
        if length != lengths[0]:
            return False
        return True

## Extraction 🧲

----

### Sedan:

Variables pool.

In [49]:
descriptions:list[str] = []
prices:list[int] = []
models:list[str] = []
years:list[int] = []
kilometers:list[int] = []
engines:list[str] = []

# seconds
DELAY:int = 1

In [50]:
sedan = resources("cars_sedan")

Get web & close initial prompt.

In [51]:
edge.get(sedan["url"])

In [52]:
close_prompt(edge)

In [53]:
elements = edge.find_elements(by=By.XPATH, value=sedan["ad"])

Extract descriptions.

In [54]:
elements[0].click()
try:
    for element in range(len(elements)):
        dynamic_delay = WebDriverWait(edge, DELAY).until(EC.presence_of_element_located((By.XPATH, sedan["description"])))
        temp = extract_description(edge, sedan["description"])
        descriptions.append(temp.text)
        click_element(edge, sedan["next"])
except TimeoutException as TE:
    descriptions.append("Web wasn't able to scrape descriptions")
except NoSuchElementException as NSE:
    click_element(edge, sedan["exit"])

Extract prices & models.

In [55]:
prices = extract_feature(edge, sedan["price"])
models = extract_feature(edge, sedan["model"])

Extract car year, engine & kilometers.

In [56]:
years = extract_feature(edge, sedan["year"])
engines = extract_feature(edge, sedan["engine"])
kilometers = extract_feature(edge, sedan["kilometers"])

In [57]:
lengths = [prices, models, years, engines, kilometers, descriptions]
get_len(lengths)

[41, 42, 42, 40]

## Cleaning 🤿

### Functions:

In [58]:
def parse_price(dataframe: pd.DataFrame, feature:str, pattern:str) -> None:
    """
    Parse financial features like prices. Works with $#,### or $#,###\n* pattern
    :param dataframe: object towards apply transformation
    :param feature: feature name towards apply transformation
    :param pattern: feature pattern to transform
    """
    dataframe[feature] = dataframe [feature] \
        .str.extract(pattern) \
        .applymap(lambda price: int(price.replace(",","")))


In [59]:
def remove_chars(dataframe: pd.DataFrame, feature:str, wildcards:str) -> pd.Series:
    """
    Remove escaped characters from features like indented large extracts of text
    :param dataframe: object towards apply transformation
    :param feature: feature name towards apply transformation
    :param wildcards: escaped characters to remove
    :return: Serie without escaped characters
    """
    return (dataframe
    .apply(
        lambda row: row[feature]
        ,axis = 1
    )
    .apply(
        lambda extracted_f: re.sub(wildcards, r' ', extracted_f)
    )
    )


In [60]:
PRICE_RG: str = r"^\$(?P<prices>\d+,\d+)?"
KM_REG: str = r"(?P<km>^\d+)"
CHARACTER_RG:str = '(\n|\r)+'

---

Remove noise elements from data list

In [61]:
# engines.append("Gasolina")

In [62]:
years.remove("Ver más")

In [63]:
models.remove("Blue Book Encuentra24")

In [64]:
# kilometers.remove("Gasolina")
# kilometers.insert(-2, "0")

In [65]:
# # engines.pop(-1)

Create dataframe to process data extracted

In [66]:
lengths = [len(engines), len(models), len(years), len(kilometers), len(prices),len(descriptions)]
lengths

[41, 41, 41, 41, 41, 40]

This was necessary due exists an announcement with no description

In [67]:
# descriptions.insert(-9, "No description")

In [68]:
# descriptions.append("No description")

In [69]:
if check_len(lengths):

    df_temp = pd.DataFrame({
        "model": models,
        "year": years,
        "kilometers": kilometers,
        "engine": engines,
        "price": prices,
        "description": descriptions
    })

    # Parse features. String to number and raw string with characters to structured string.
    parse_price(df_temp, "price", PRICE_RG)
    df_temp["kilometers"] = df_temp["kilometers"].str.extract(KM_REG).applymap(int)
    df_temp["year"] = df_temp["year"].map(int)
    df_temp["description"] = remove_chars(df_temp, "description", CHARACTER_RG)

    # Export dataframe in .csv format
    persistence(df_temp)

In [70]:
df_temp

,model,year,kilometers,engine,price,description
0,Mitsubishi Mirage G4,2021,3000,Gasolina,12600,Color : Azul DESCRIPCIÓN CALIFICACIONES SEGURI...
1,Nissan Versa,2017,78000,Gasolina,7400,Financiamiento: si DESCRIPCIÓN CALIFICACIONES ...
2,Kia Forte,2016,34000,Gasolina,6900,Color : Blanco Se vende Mitsubishi G4 2018. Au...
3,Mitsubishi Mirage G4,2018,60000,Gasolina,8500,Color : Azul Vendo Mitsubishi Lancer 2017 Full...
4,Mitsubishi Lancer,2017,68000,Gasolina,8700,Precio negociable: si Vendo Honda Civid 2018 E...
5,Honda Civic,2018,30000,Gasolina,15000,Garantía: Como es visto no hay garantía Precio...
6,Toyota Corolla,1999,254128,Gasolina,4000,Color : Negro DESCRIPCIÓN CALIFICACIONES SEGUR...
7,Nissan Sentra,2014,98000,Gasolina,7400,Financiamiento: no Precio negociable: si Color...
8,Nissan Sentra,2017,109435,Gasolina,7500,Color : Negro Bonito Mitsubishi Mirage G4 2021...
9,Mitsubishi Mirage G4,2021,4000,Gasolina,10000,Color : Gris Se vende Kia Río 2020. Automático...
